<a href="https://colab.research.google.com/github/LoganToullec/ProgGPU/blob/main/Copie_de_ProgGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook
This notebook is a starting point to setup for CUDA code.
If you want to save your codes, you will have to create a copy of this notebook, and then work on this copy

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


# Prepare your notebook to handle CUDA code

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-h0v9wx7z
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-h0v9wx7z
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.1.0-py3-none-any.whl size=8011 sha256=e2b2acbb79d726227ace2ffdeb18b901e4debb9cf9419b13a35a389a655ee4ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-nz384yph/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Source files will be saved in "/tmp/tmpzboh6w5t".


# Install & Use of CCfits for TD 2


In [ ]:
!apt-get install libccfits-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libccfits0v5
Suggested packages:
  libccfits-doc
The following NEW packages will be installed:
  libccfits-dev libccfits0v5
0 upgraded, 2 newly installed, 0 to remove and 38 not upgraded.
Need to get 501 kB of archives.
After this operation, 3,105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libccfits0v5 amd64 2.6+dfsg-1 [195 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libccfits-dev amd64 2.6+dfsg-1 [306 kB]
Fetched 501 kB in 0s (1,111 kB/s)
Selecting previously unselected package libccfits0v5:amd64.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../libccfits0v5_2.6+dfsg-1_amd64.deb ...
Unpacking libccfits0v5:amd64 (2.6+dfsg-1) ...
Selecting previously unselected package libccfits-dev:amd64.
Preparing to unpack ...

In [ ]:
import astropy.io.fits as pf
import numpy as np

A = np.random.random((10,10))
pf.writeto("test.fits", A, overwrite=True)

In [ ]:
%%writefile test2.cu
#include <stdio.h>
#include <CCfits>

int main(void) {
    CCfits::FITS inFile("test.fits", CCfits::Read, true);
    CCfits::PHDU &phdu = inFile.pHDU();
    std::valarray<float> fitsImage;
    phdu.read(fitsImage);
    std::cout << fitsImage[1] << std::endl;
    std::cout << fitsImage.size() << std::endl;
    long naxes[2] = {10,10};
    CCfits::FITS *inFile2 = new CCfits::FITS("test2.fits", FLOAT_IMG, 2, naxes);
    CCfits::PHDU &phdu2 = inFile2->pHDU();
    phdu2.write(1, 99, fitsImage);
}


Writing test2.cu


In [ ]:
!nvcc test2.cu -o test2 -I/usr/include/CCfits -L/usr/lib/x86_64-linux-gnu/ -std=c++11 -lCCfits -lcfitsio

In [ ]:
!./test2

0.43347
100


In [ ]:
import astropy.io.fits as pf
import numpy as np
A = pf.getdata("test2.fits")
B = pf.getdata("test.fits")

In [ ]:
!conda install -c anaconda libpng

/bin/bash: line 1: conda: command not found


# Code with cuda

In [ ]:
%%cuda
#include <iostream>
#include <fstream>
#include <cstdint>
#include <cmath>
#include <complex>
#include <cuda_runtime.h>

// Définir les dimensions de l'image pour chaque fractale
const int WIDTH = 30000;
const int HEIGHT = 30000;

// Définir les régions des ensembles de fractales
const double M_MIN_X = -2.0;
const double M_MAX_X = 1.0;
const double M_MIN_Y = -1.5;
const double M_MAX_Y = 1.5;

const double J_MIN_X = -1.5;
const double J_MAX_X = 1.5;
const double J_MIN_Y = -1.5;
const double J_MAX_Y = 1.5;
const std::complex<double> C(-0.8, 0.156); // Paramètre de Julia

const double BS_MIN_X = -2.0;
const double BS_MAX_X = 1.0;
const double BS_MIN_Y = -2.0;
const double BS_MAX_Y = 2.0;

const double N_MIN_X = -1.0;
const double N_MAX_X = 1.0;
const double N_MIN_Y = -1.0;
const double N_MAX_Y = 1.0;

const double MB_MIN_X = -2.0;
const double MB_MAX_X = 2.0;
const double MB_MIN_Y = -2.0;
const double MB_MAX_Y = 2.0;
const int MB_POWER = 4; // Puissance pour le Multibrot

const double T_MIN_X = -2.0;
const double T_MAX_X = 2.0;
const double T_MIN_Y = -2.0;
const double T_MAX_Y = 2.0;

// Fonction CUDA pour évaluer l'ensemble de Mandelbrot
__device__ int mandelbrot(double cx, double cy, int max_iterations) {
    double x = 0.0, y = 0.0;
    for (int i = 0; i < max_iterations; ++i) {
        double temp = x * x - y * y + cx;
        y = 2.0 * x * y + cy;
        x = temp;
        if (x * x + y * y > 4.0) return i;
    }
    return max_iterations;
}

// Fonction CUDA pour évaluer l'ensemble de Julia
__device__ int julia(double x, double y, int max_iterations) {
    for (int i = 0; i < max_iterations; ++i) {
        double temp = x * x - y * y + -0.8;
        y = 2.0 * x * y + 0.156;
        x = temp;
        if (x * x + y * y > 4.0) return i;
    }
    return max_iterations;
}

// Fonction CUDA pour évaluer l'ensemble de Burning Ship
__device__ int burningShip(double cx, double cy, int max_iterations) {
    double x = 0.0, y = 0.0;
    for (int i = 0; i < max_iterations; ++i) {
        double temp = x * x - y * y + cx;
        y = fabs(2.0 * x * y) + cy;
        x = temp;
        if (x * x + y * y > 4.0) return i;
    }
    return max_iterations;
}

// Fonction CUDA pour évaluer l'ensemble de Newton
__device__ int newton(double x, double y, int max_iterations) {
    for (int i = 0; i < max_iterations; ++i) {
        double temp = x * x * x - 3.0 * x * y * y - 1.0;
        double temp_y = 3.0 * x * x * y - y * y * y;
        x = x - (x * temp - y * temp_y) / (3.0 * (x * x - y * y));
        y = y - (y * temp + x * temp_y) / (3.0 * (x * x - y * y));
        if (x * x + y * y < 1e-6) return i;
    }
    return max_iterations;
}

// Fonction CUDA pour évaluer l'ensemble de Multibrot
__device__ int multibrot(double cx, double cy, int max_iterations, int power) {
    double x = 0.0, y = 0.0;
    for (int i = 0; i < max_iterations; ++i) {
        double temp = pow(x * x + y * y, power / 2.0) * cos(power * atan2(y, x)) + cx;
        y = pow(x * x + y * y, power / 2.0) * sin(power * atan2(y, x)) + cy;
        x = temp;
        if (x * x + y * y > 4.0) return i;
    }
    return max_iterations;
}

// Fonction CUDA pour évaluer l'ensemble de Tricorn
__device__ int tricorn(double cx, double cy, int max_iterations) {
    double x = 0.0, y = 0.0;
    for (int i = 0; i < max_iterations; ++i) {
        double temp = x * x - y * y + cx;
        y = -2.0 * x * y + cy;
        x = temp;
        if (x * x + y * y > 4.0) return i;
    }
    return max_iterations;
}

// Kernel CUDA pour générer une fractale
__global__ void generateFractalCUDA(uint8_t* d_image, double min_x, double max_x, double min_y, double max_y, int max_iterations, int fractalType, int power) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    if (x < WIDTH && y < HEIGHT) {
        double cx = min_x + (max_x - min_x) * x / WIDTH;
        double cy = min_y + (max_y - min_y) * y / HEIGHT;
        int index = y * WIDTH + x;
        switch (fractalType) {
            case 0:
                d_image[index] = mandelbrot(cx, cy, max_iterations);
                break;
            case 1:
                d_image[index] = julia(cx, cy, max_iterations);
                break;
            case 2:
                d_image[index] = burningShip(cx, cy, max_iterations);
                break;
            case 3:
                d_image[index] = newton(cx, cy, max_iterations);
                break;
            case 4:
                d_image[index] = multibrot(cx, cy, max_iterations, power);
                break;
            case 5:
                d_image[index] = tricorn(cx, cy, max_iterations);
                break;
        }
        // Coloration unicolor avec variations d'opacité
        double opacity = 1.0 - static_cast<double>(d_image[index]) / max_iterations;
        uint8_t color = 255; // Couleur unie blanche
        d_image[index] = static_cast<uint8_t>(color * opacity);
    }
}

// Fonction pour générer et enregistrer une image de fractale
void generateFractal(const char* filename, double min_x, double max_x, double min_y, double max_y, int max_iterations, int fractalType, int power) {
    uint8_t* h_image = new uint8_t[WIDTH * HEIGHT];
    uint8_t* d_image;
    cudaMalloc(&d_image, WIDTH * HEIGHT * sizeof(uint8_t));

    dim3 blockSize(16, 16); // Taille du bloc CUDA
    dim3 gridSize((WIDTH + blockSize.x - 1) / blockSize.x, (HEIGHT + blockSize.y - 1) / blockSize.y);

    generateFractalCUDA<<<gridSize, blockSize>>>(d_image, min_x, max_x, min_y, max_y, max_iterations, fractalType, power);

    cudaMemcpy(h_image, d_image, WIDTH * HEIGHT * sizeof(uint8_t), cudaMemcpyDeviceToHost);

    std::ofstream outfile(filename, std::ios::binary);
    outfile << "BM";
    int fileSize = WIDTH * HEIGHT * 3 + 54; // Taille totale du fichier BMP
    int paddingSize = (4 - (WIDTH * 3) % 4) % 4; // Calcul de la taille du padding
    int dataSize = (WIDTH * 3 + paddingSize) * HEIGHT; // Taille des données de l'image
    int totalSize = fileSize + dataSize; // Taille totale du fichier
    outfile.write(reinterpret_cast<char*>(&totalSize), sizeof(int));
    int reserved = 0;
    outfile.write(reinterpret_cast<char*>(&reserved), sizeof(int));
    int offset = 54;
    outfile.write(reinterpret_cast<char*>(&offset), sizeof(int));
    int headerSize = 40;
    outfile.write(reinterpret_cast<char*>(&headerSize), sizeof(int));
    int width = WIDTH;
    outfile.write(reinterpret_cast<char*>(&width), sizeof(int));
    int height = HEIGHT;
    outfile.write(reinterpret_cast<char*>(&height), sizeof(int));
    short planes = 1;
    outfile.write(reinterpret_cast<char*>(&planes), sizeof(short));
    short bpp = 24;
    outfile.write(reinterpret_cast<char*>(&bpp), sizeof(short));
    int compression = 0;
    outfile.write(reinterpret_cast<char*>(&compression), sizeof(int));
    outfile.write(reinterpret_cast<char*>(&dataSize), sizeof(int));
    int resolutionX = 2835; // 72 dpi
    outfile.write(reinterpret_cast<char*>(&resolutionX), sizeof(int));
    int resolutionY = 2835; // 72 dpi
    outfile.write(reinterpret_cast<char*>(&resolutionY), sizeof(int));
    int colors = 0;
    outfile.write(reinterpret_cast<char*>(&colors), sizeof(int));
    int importantColors = 0;
    outfile.write(reinterpret_cast<char*>(&importantColors), sizeof(int));

    for (int y = HEIGHT - 1; y >= 0; --y) {
        for (int x = 0; x < WIDTH; ++x) {
            uint8_t value = h_image[y * WIDTH + x];
            outfile.write(reinterpret_cast<char*>(&value), sizeof(uint8_t));
            outfile.write(reinterpret_cast<char*>(&value), sizeof(uint8_t));
            outfile.write(reinterpret_cast<char*>(&value), sizeof(uint8_t));
        }
        // Écriture du padding
        for (int p = 0; p < paddingSize; ++p) {
            uint8_t paddingByte = 0;
            outfile.write(reinterpret_cast<char*>(&paddingByte), sizeof(uint8_t));
        }
    }

    outfile.close();
    cudaFree(d_image);
    delete[] h_image;
    std::cout << "Image " << filename << " générée avec succès." << std::endl;
}

int main() {
    //generateFractal("fractals_cuda/mandelbrot.bmp", M_MIN_X, M_MAX_X, M_MIN_Y, M_MAX_Y, 1000, 0, 0);
    generateFractal("fractals_cuda/julia.bmp", J_MIN_X, J_MAX_X, J_MIN_Y, J_MAX_Y, 500, 1, 0);
    //generateFractal("fractals_cuda/burning_ship.bmp", BS_MIN_X, BS_MAX_X, BS_MIN_Y, BS_MAX_Y, 1000, 2, 0);
    //generateFractal("fractals_cuda/multibrot.bmp", MB_MIN_X, MB_MAX_X, MB_MIN_Y, MB_MAX_Y, 1000, 4, MB_POWER);
    //generateFractal("fractals_cuda/tricorn.bmp", T_MIN_X, T_MAX_X, T_MIN_Y, T_MAX_Y, 1000, 5, 0);

    return 0;
}

Image fractals_cuda/julia.bmp générée avec succès.



# Without using CUDA

In [ ]:
%%cuda
#include <iostream>
#include <fstream>
#include <cstdint>
#include <cmath>
#include <complex>
#include <chrono>

// Définir les dimensions de l'image pour chaque fractale
const int WIDTH = 10000;
const int HEIGHT = 10000;

// Définir les régions des ensembles de fractales
const double M_MIN_X = -2.0;
const double M_MAX_X = 1.0;
const double M_MIN_Y = -1.5;
const double M_MAX_Y = 1.5;

const double J_MIN_X = -1.5;
const double J_MAX_X = 1.5;
const double J_MIN_Y = -1.5;
const double J_MAX_Y = 1.5;
const std::complex<double> C(-0.8, 0.156); // Paramètre de Julia

const double BS_MIN_X = -2.0;
const double BS_MAX_X = 1.0;
const double BS_MIN_Y = -2.0;
const double BS_MAX_Y = 2.0;

const double N_MIN_X = -1.0;
const double N_MAX_X = 1.0;
const double N_MIN_Y = -1.0;
const double N_MAX_Y = 1.0;

const double MB_MIN_X = -2.0;
const double MB_MAX_X = 2.0;
const double MB_MIN_Y = -2.0;
const double MB_MAX_Y = 2.0;
const int MB_POWER = 4; // Puissance pour le Multibrot

const double T_MIN_X = -2.0;
const double T_MAX_X = 2.0;
const double T_MIN_Y = -2.0;
const double T_MAX_Y = 2.0;

// Fonction pour évaluer l'ensemble de Mandelbrot
int mandelbrot(double cx, double cy, int max_iterations) {
    double x = 0.0, y = 0.0;
    for (int i = 0; i < max_iterations; ++i) {
        double temp = x * x - y * y + cx;
        y = 2.0 * x * y + cy;
        x = temp;
        if (x * x + y * y > 4.0) return i;
    }
    return max_iterations;
}

// Fonction pour évaluer l'ensemble de Julia
int julia(double x, double y, int max_iterations) {
    for (int i = 0; i < max_iterations; ++i) {
        double temp = x * x - y * y + -0.8;
        y = 2.0 * x * y + 0.156;
        x = temp;
        if (x * x + y * y > 4.0) return i;
    }
    return max_iterations;
}

// Fonction pour évaluer l'ensemble de Burning Ship
int burningShip(double cx, double cy, int max_iterations) {
    double x = 0.0, y = 0.0;
    for (int i = 0; i < max_iterations; ++i) {
        double temp = x * x - y * y + cx;
        y = fabs(2.0 * x * y) + cy;
        x = temp;
        if (x * x + y * y > 4.0) return i;
    }
    return max_iterations;
}

// Fonction pour évaluer l'ensemble de Newton
int newton(double x, double y, int max_iterations) {
    for (int i = 0; i < max_iterations; ++i) {
        double temp = x * x * x - 3.0 * x * y * y - 1.0;
        double temp_y = 3.0 * x * x * y - y * y * y;
        x = x - (x * temp - y * temp_y) / (3.0 * (x * x - y * y));
        y = y - (y * temp + x * temp_y) / (3.0 * (x * x - y * y));
        if (x * x + y * y < 1e-6) return i;
    }
    return max_iterations;
}

// Fonction pour évaluer l'ensemble de Multibrot
int multibrot(double cx, double cy, int max_iterations, int power) {
    double x = 0.0, y = 0.0;
    for (int i = 0; i < max_iterations; ++i) {
        double temp = pow(x * x + y * y, power / 2.0) * cos(power * atan2(y, x)) + cx;
        y = pow(x * x + y * y, power / 2.0) * sin(power * atan2(y, x)) + cy;
        x = temp;
        if (x * x + y * y > 4.0) return i;
    }
    return max_iterations;
}

// Fonction pour évaluer l'ensemble de Tricorn
int tricorn(double cx, double cy, int max_iterations) {
    double x = 0.0, y = 0.0;
    for (int i = 0; i < max_iterations; ++i) {
        double temp = x * x - y * y + cx;
        y = -2.0 * x * y + cy;
        x = temp;
        if (x * x + y * y > 4.0) return i;
    }
    return max_iterations;
}

// Fonction pour générer et enregistrer une image de fractale
void generateFractal(const char* filename, double min_x, double max_x, double min_y, double max_y, int max_iterations, int fractalType, int power) {
    uint8_t* h_image = new uint8_t[WIDTH * HEIGHT];

    auto start = std::chrono::high_resolution_clock::now();

    for (int y = 0; y < HEIGHT; ++y) {
        for (int x = 0; x < WIDTH; ++x) {
            double cx = min_x + (max_x - min_x) * x / WIDTH;
            double cy = min_y + (max_y - min_y) * y / HEIGHT;
            int index = y * WIDTH + x;
            switch (fractalType) {
                case 0:
                    h_image[index] = mandelbrot(cx, cy, max_iterations);
                    break;
                case 1:
                    h_image[index] = julia(cx, cy, max_iterations);
                    break;
                case 2:
                    h_image[index] = burningShip(cx, cy, max_iterations);
                    break;
                case 3:
                    h_image[index] = newton(cx, cy, max_iterations);
                    break;
                case 4:
                    h_image[index] = multibrot(cx, cy, max_iterations, power);
                    break;
                case 5:
                    h_image[index] = tricorn(cx, cy, max_iterations);
                    break;
            }
            // Coloration unicolor avec variations d'opacité
            double opacity = 1.0 - static_cast<double>(h_image[index]) / max_iterations;
            uint8_t color = 255; // Couleur unie blanche
            h_image[index] = static_cast<uint8_t>(color * opacity);
        }
    }

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_seconds = end - start;
    std::cout << "Temps d'exécution : " << elapsed_seconds.count() << " secondes." << std::endl;

    std::ofstream outfile(filename, std::ios::binary);
    outfile << "BM";
    int fileSize = WIDTH * HEIGHT * 3 + 54;
    outfile.write(reinterpret_cast<char*>(&fileSize), sizeof(int));
    int reserved = 0;
    outfile.write(reinterpret_cast<char*>(&reserved), sizeof(int));
    int offset = 54;
    outfile.write(reinterpret_cast<char*>(&offset), sizeof(int));
    int headerSize = 40;
    outfile.write(reinterpret_cast<char*>(&headerSize), sizeof(int));
    int width = WIDTH;
    outfile.write(reinterpret_cast<char*>(&width), sizeof(int));
    int height = HEIGHT;
    outfile.write(reinterpret_cast<char*>(&height), sizeof(int));
    short planes = 1;
    outfile.write(reinterpret_cast<char*>(&planes), sizeof(short));
    short bpp = 24;
    outfile.write(reinterpret_cast<char*>(&bpp), sizeof(short));
    int compression = 0;
    outfile.write(reinterpret_cast<char*>(&compression), sizeof(int));
    int imgSize = WIDTH * HEIGHT * 3;
    outfile.write(reinterpret_cast<char*>(&imgSize), sizeof(int));
    int resolutionX = 2835;
    outfile.write(reinterpret_cast<char*>(&resolutionX), sizeof(int));
    int resolutionY = 2835;
    outfile.write(reinterpret_cast<char*>(&resolutionY), sizeof(int));
    int colors = 0;
    outfile.write(reinterpret_cast<char*>(&colors), sizeof(int));
    int importantColors = 0;
    outfile.write(reinterpret_cast<char*>(&importantColors), sizeof(int));

    for (int y = HEIGHT - 1; y >= 0; --y) {
        for (int x = 0; x < WIDTH; ++x) {
            uint8_t value = h_image[y * WIDTH + x];
            outfile.write(reinterpret_cast<char*>(&value), sizeof(uint8_t));
            outfile.write(reinterpret_cast<char*>(&value), sizeof(uint8_t));
            outfile.write(reinterpret_cast<char*>(&value), sizeof(uint8_t));
        }
    }

    outfile.close();
    delete[] h_image;
    std::cout << "Image " << filename << " générée avec succès." << std::endl;
}

int main() {
    //generateFractal("fractals/mandelbrot.bmp", M_MIN_X, M_MAX_X, M_MIN_Y, M_MAX_Y, 1000, 0, 0);
    generateFractal("fractals/julia.bmp", J_MIN_X, J_MAX_X, J_MIN_Y, J_MAX_Y, 500, 1, 0);
    //generateFractal("fractals/burning_ship.bmp", BS_MIN_X, BS_MAX_X, BS_MIN_Y, BS_MAX_Y, 1000, 2, 0);
    //generateFractal("fractals/multibrot.bmp", MB_MIN_X, MB_MAX_X, MB_MIN_Y, MB_MAX_Y, 1000, 4, MB_POWER);
    //generateFractal("fractals/tricorn.bmp", T_MIN_X, T_MAX_X, T_MIN_Y, T_MAX_Y, 1000, 5, 0);

    return 0;
}


Temps d'exécution : 23.9182 secondes.
Image fractals/julia.bmp générée avec succès.

